In [196]:
import pandas as pd
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import logging
from tqdm import tqdm
import requests
import random
import time
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv # For loading .env file
import shutil
import re
from pandas import Period
import glob
import numpy as np

In [24]:
# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='json_to_csv.log',
    filemode='a'
)
logger = logging.getLogger()

In [122]:
# Tạo danh sách username
list_kol = pd.read_excel('List_KOL_final.xlsx', sheet_name='Sheet1')
# Lọc DataFrame để chỉ lấy các hàng mà cột 'erdeleted_kolror' không phải là null
kol_with_errors = list_kol[list_kol['deleted_kol'].isna()]
kol_usernames = kol_with_errors["name"].tolist()

In [26]:
json_files = [name + '.json' for name in kol_usernames]

In [27]:
len(json_files)

82

In [28]:
# Danh sách tên cột cần lấy
columns = [
    'text', 'createTime', 'createTimeISO', 'isAd', 'profileUrl',
    'musicName', 'musicId', 'webVideoUrl', 'duration', 'diggCount',
    'shareCount', 'playCount', 'collectCount', 'commentCount',
    'hashtags', 'isSlideshow', 'isSponsored', 'kol_username',
    'ttSeller', 'commerceUser'
]

In [34]:
json_files[1:82]

['hannaholala.json',
 'goc.cua.ru.json',
 'mai.trinh.h.json',
 'tomskincare.json',
 'rosermae.json',
 'chouchinchan.json',
 'nguyenhuemakeup1996.json',
 'trinhpham2222.json',
 'bylinhtruong.json',
 'maivantrang_.json',
 'linhtruong.ld.json',
 'ngoctrinh89.json',
 'dramakinglndx.json',
 'msquynhthie.json',
 'truc.dao.json',
 'dieplamanh.json',
 'lehatruc.daily.json',
 'bychloenguyen.json',
 'im.lunadao_official.json',
 'hoangminhngoc21.json',
 'thaonhileofficial.json',
 'actress.lanhuong.json',
 'blingbabi_.json',
 'hwitch99.json',
 'emmi.hoang.json',
 'thaovann_0602.json',
 'chaubui.json',
 'nhatkycuameichan.json',
 'tnhadinh.json',
 'thudannguyen99.json',
 'thanhyvo.json',
 'ph.hoanganhh.json',
 'myhuyen_channel.json',
 'vznganmin.json',
 'rinavaseoul2003.json',
 'tran_lam18.json',
 'biveve131.json',
 'tikkaisweird.json',
 'jjhahehi.json',
 'khietnhinhi.json',
 'beautytips4.0.json',
 'gigigithebabe.json',
 'hufureview.json',
 '_inlil.json',
 'thangmakeup.json',
 'trishphann.json',
 'h

In [33]:
def safe_get(d, keys, default=None):
    for key in keys:
        if isinstance(d, dict) and key in d:
            d = d[key]
        else:
            return default
    return d

def process_json_file(json_file):
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        logger.error(f"Không thể mở hoặc parse file {json_file}: {e}")
        return

    rows = []
    for video in data:
        try:
            create_time = safe_get(video, ['createTime'])
            create_time_iso = safe_get(video, ['createTimeISO'])

            if create_time:
                dt_vn = datetime.fromtimestamp(create_time, tz=timezone.utc) + timedelta(hours=7)
                create_time_str = dt_vn.strftime('%Y-%m-%d %H:%M:%S')
            else:
                create_time_str = None

            if create_time_iso:
                dt_iso = datetime.fromisoformat(create_time_iso.replace('Z', '+00:00'))
                create_time_iso_str = dt_iso.strftime('%d/%m/%Y')
            else:
                create_time_iso_str = None

            hashtags_list = safe_get(video, ['hashtags'], [])
            hashtags = ', '.join(tag.get('name', '') for tag in hashtags_list) if hashtags_list else None

            row = {
                'text': safe_get(video, ['text']),
                'createTime': create_time_str,
                'createTimeISO': create_time_iso_str,
                'isAd': safe_get(video, ['isAd']),
                'profileUrl': safe_get(video, ['authorMeta', 'profileUrl']),
                'musicName': safe_get(video, ['musicMeta', 'musicName']),
                'musicId': safe_get(video, ['musicMeta', 'musicId']),
                'webVideoUrl': safe_get(video, ['webVideoUrl']),
                'duration': safe_get(video, ['videoMeta', 'duration']),
                'diggCount': safe_get(video, ['diggCount']),
                'shareCount': safe_get(video, ['shareCount']),
                'playCount': safe_get(video, ['playCount']),
                'collectCount': safe_get(video, ['collectCount']),
                'commentCount': safe_get(video, ['commentCount']),
                'hashtags': hashtags,
                'isSlideshow': safe_get(video, ['isSlideshow']),
                'isSponsored': safe_get(video, ['isSponsored']),
                'kol_username': safe_get(video, ['kol_username']),
                'ttSeller': safe_get(video, ['authorMeta', 'ttSeller']),
                'commerceUser': safe_get(video, ['authorMeta', 'commerceUserInfo', 'commerceUser'])
            }

            rows.append(row)

        except Exception as e:
            logger.error(f"Lỗi khi xử lý video trong {json_file}: {e}")
            continue

    df = pd.DataFrame(rows, columns=columns)
    file_stem = os.path.splitext(os.path.basename(json_file))[0]
    csv_path = f"./kol_data_csv/{file_stem}.csv"
    try:
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        logger.info(f"Đã lưu thành công: {csv_path}")
    except Exception as e:
        logger.error(f"Không thể lưu {csv_path}: {e}")

In [35]:
# Chạy toàn bộ file
for json_file in json_files[1:82]:
    if os.path.exists(json_file):
        process_json_file(json_file)
    else:
        logger.warning(f"File không tồn tại: {json_file}")

---

In [39]:
df_check = pd.read_csv('./kol_data_csv/halinhofficial.csv', parse_dates=["createTime", "createTimeISO"])

In [45]:
df_check_exolyt = pd.read_csv('./Crawl_exolyt/halinhofficial.csv', parse_dates=['Date'])
df_check_exolyt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   526 non-null    int64         
 1   Date         526 non-null    datetime64[ns]
 2   Exact data*  94 non-null     object        
 3   Followers    526 non-null    object        
 4   Followers+   526 non-null    object        
 5   Views        526 non-null    object        
 6   Views+       526 non-null    object        
 7   Likes        526 non-null    object        
 8   Likes+       526 non-null    object        
 9   Videos       526 non-null    int64         
 10  Videos+      526 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 45.3+ KB


In [51]:
df_check['createTimeISO']

0      10/06/2025
1      06/06/2025
2      05/06/2025
3      04/06/2025
4      02/06/2025
          ...    
201    17/01/2024
202    12/01/2024
203    10/01/2024
204    08/01/2024
205    02/01/2024
Name: createTimeISO, Length: 206, dtype: object

In [46]:
df_check_exolyt['Date'] 

0     2025-06-09
1     2025-06-08
2     2025-06-07
3     2025-06-06
4     2025-06-05
         ...    
521   2024-01-05
522   2024-01-04
523   2024-01-03
524   2024-01-02
525   2024-01-01
Name: Date, Length: 526, dtype: datetime64[ns]

In [ ]:
# hàm chuyển đổi follower+ từ object thành int
def convert_signed_follower_value(s):
    if pd.isna(s):
        return 0.0
    s = s.strip().upper()
    sign = -1 if s.startswith('-') else 1
    s = s.lstrip('+-')  # Bỏ dấu '+' hoặc '-'
    
    if 'K' in s:
        return sign * float(s.replace('K', '')) * 1_000
    elif 'M' in s:
        return sign * float(s.replace('M', '')) * 1_000_000
    else:
        return sign * float(s)

---

# Tạo thuộc tính nhận biết caption có liên quan đến các trend gần đây không ?

In [100]:
# === Logging setup ===
logger_gemini = logging.getLogger('gemini')

# Xóa tất cả các handler cũ (không ảnh hưởng đến file log)
for handler in logger_gemini.handlers[:]:
    logger_gemini.removeHandler(handler)

# Gắn lại handler mới, vẫn ghi tiếp vào file cũ (append mode)
handler = logging.FileHandler('gemini.log', mode='a')  # mode='a' là mặc định, vẫn nên ghi rõ
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger_gemini.addHandler(handler)
logger_gemini.setLevel(logging.INFO)


# === Load API keys ===
if load_dotenv(find_dotenv('secret.env')):
    logger_gemini.info("✅ Loaded environment variables from secret.env")
else:
    logger_gemini.warning("⚠️ Could not find secret.env, using current environment variables")

API_KEYS = [os.getenv(f"API_GEMINI_{i}") for i in range(1, 12)]
API_KEYS = [key for key in API_KEYS if key]  # Lọc key hợp lệ

if not API_KEYS:
    logger_gemini.error("❌ Không tìm thấy API key nào hợp lệ.")
    raise ValueError("Không có API key hợp lệ trong file secret.env")

current_key_index = 0
current_key = API_KEYS[current_key_index]
genai.configure(api_key=current_key)

# === Request Tracking ===
request_counter = 0
last_minute = datetime.now()
daily_request_counter = 0
DAILY_LIMIT = 1500
RPM_LIMIT = 30

In [94]:
TEMPERATURE = 0.2
model = genai.GenerativeModel("gemini-2.0-flash-lite")

In [95]:
retry_on_rate_limit = 0  # Đếm số lần retry 1 tiếng
MAX_RETRY_WAIT_HOURS = 19
just_rotated = False

def rotate_key():
    global current_key_index, current_key, retry_on_rate_limit, model, just_rotated
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    current_key = API_KEYS[current_key_index]
    genai.configure(api_key=current_key)
    model = genai.GenerativeModel("gemini-2.0-flash-lite")  # <- cập nhật model tương ứng với key mới
    retry_on_rate_limit = 0
    daily_request_counter = 1  # reset về 1 ngay khi rotate
    just_rotated = True  # <== Đánh dấu vừa mới xoay key
    logger_gemini.info(f"🔁 Đã chuyển sang API Key #{current_key_index + 1}")


def call_gemini(prompt, caption, retries=3):
    global request_counter, last_minute, daily_request_counter, model, just_rotated

    try:
        response = model.generate_content(
            prompt,
            generation_config={"temperature": TEMPERATURE}
        )
        request_counter += 1
        daily_request_counter += 1
        just_rotated = False  # <== Reset flag nếu gọi thành công
        logger_gemini.info(f"✅ Thành công (#{daily_request_counter}): Caption = \"{caption[:80]}\" -> Result: {response.text.strip()}")
        time.sleep(2.2)  # Đảm bảo không vượt giới hạn sliding window
        return response.text

    except Exception as e:
        if retries > 0 and ("quota" in str(e).lower() or "rate" in str(e).lower()):
            logger_gemini.warning(f"⚠️ Rate limit với API Key hiện tại: {str(e)}")
            rotate_key()
            return call_gemini(prompt, caption, retries - 1)
        else:
            logger_gemini.error(f"❌ Lỗi không xử lý được: {str(e)}")
            raise

In [96]:
# Hàm tạo prompt từ caption và ngày
def build_prompt(caption, date_str):
    return f"""
You are a social media caption analyst.

You will be given a caption from a TikTok post by a Vietnamese KOL (Key Opinion Leader) who primarily promotes beauty and cosmetic products.  
The audience is mainly Vietnamese, and the caption may or may not be related to popular TikTok trends, holidays, or events — especially those relevant in Vietnam, but also possibly international.
You should only consider trends or events that occur within **2 weeks before or after** the given date.

Your task is to determine whether, given the date, the caption is contextually related to any **ongoing or upcoming** TikTok trend, holiday, or event.

If it is related, respond with **'1'**.  
If it is not related, respond with **'0'**.  
Only respond with a single character: '1' or '0'.

Caption: "{caption}"  
Date posted: {date_str}
"""

In [97]:
# Hàm gọi Gemini kèm retry nếu lỗi
def get_result_with_retry(prompt, caption, max_retries=3):
    attempt = 0
    while attempt < max_retries:
        try:
            result = call_gemini(prompt, caption)
            cleaned = result.strip().lower()
            return True if cleaned == '1' else False
        except Exception as e:
            attempt += 1
            logger_gemini.warning(
                f"⚠️ Lỗi khi gọi Gemini (lần {attempt}) cho caption: {caption[:30]}... | Error: {str(e)}"
            )
            time.sleep(2.2)  # Chờ 2 giây rồi thử lại
    # Nếu hết lượt retry mà vẫn lỗi
    logger_gemini.error(f"❌ Caption thất bại sau {max_retries} lần: {caption[:30]}...")
    return 'error'

In [85]:
# tên các file csv
csv_files = [name + '.csv' for name in kol_usernames]

In [99]:
csv_files[61:]

['lethikhanhhuyen2004.csv',
 'im.lukedo.csv',
 'emma1010_official.csv',
 'luongthuchien6868.csv',
 'michanne1011.csv',
 'vithanhle308.csv',
 'rose_do18.csv',
 'huynhanhthu_cosmetic.csv',
 'alissaho23.csv',
 'itshale98.csv',
 'kimchungphan20.csv',
 'haleypham2002.csv',
 'dieplinhcami.csv',
 'eb06thg9.csv',
 'han.hang18.csv',
 'annie.rose697.csv',
 'drkhonghanhnguyen.csv',
 'phoganh8461.csv',
 'phuonglytran.csv',
 'xukamechamda.csv',
 'hienlamdep90.csv']

In [90]:
len(csv_files[60:])

22

In [101]:
input_folder = './kol_data_csv/'
output_folder = './kol_data_with_istrend/'

# Tạo thư mục đầu ra nếu chưa có
os.makedirs(output_folder, exist_ok=True)

for file_name in csv_files[61:]:
    file_path = os.path.join(input_folder, file_name)
    output_path = os.path.join(output_folder, file_name)

    logger_gemini.info(f"📂 Đang xử lý file: {file_path}")
    try:
        df = pd.read_csv(file_path)
        results = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {file_name}"):
            caption = str(row['text']) if pd.notna(row['text']) else ''
            date_str = str(row['createTimeISO']) if pd.notna(row['createTimeISO']) else ''
            prompt = build_prompt(caption, date_str)

            result = get_result_with_retry(prompt, caption)

            results.append(result)

        df['is_trending'] = results

        # ✅ Ghi vào thư mục output mới
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        logger_gemini.info(f"✅ Đã xử lý xong và lưu tại: {output_path}")

    except Exception as e:
        logger_gemini.error(f"❌ Lỗi khi xử lý file {file_path}: {str(e)}")


Processing hienlamdep90.csv: 100%|██████████| 255/255 [12:52<00:00,  3.03s/it]


In [ ]:
# đang bị thừa chỗ limit request per day. Dẫn đến có vài api chưa chạy hết được khả năng mà đã bị chuyển sang api khác.
# => Ta vẫn sẽ giữ lại biến điếm số request mỗi api. Tuy nhiên không chuyển api khi đạt >= 1500 nữa mà tự đổi luôn. Và khi đổi thì cập nhật lại số
# request = 0.
# => Bây giờ chờ duyệt qua hết 10 api_key rồi sẽ sửa lại code và 7h sáng mai cào tiếp ( cho đảm bảo đồng bộ tất cả theo mục tiêu ban đầu ) 

---

# Tạo thêm các thuộc tính khác

In [116]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map
    "\U0001F1E0-\U0001F1FF"  # flags
    "\U00002700-\U000027BF"  # dingbats
    "\U0001F900-\U0001F9FF"  # supplemental symbols
    "\U00002600-\U000026FF"  # miscellaneous symbols
    "\U00002B50"
    "\U0001FA70-\U0001FAFF"  # extended symbols
    "\U0001F700-\U0001F77F"
    "]+", flags=re.UNICODE
)

In [117]:
# 1. Đếm emoji
def count_emojis(text):
    return len(emoji_pattern.findall(text))

# 2. Đếm hashtag
def count_hashtags(text):
    return len(re.findall(r"#\w+", text))

# 3. Bỏ emoji và hashtag
def remove_emojis(text):
    return emoji_pattern.sub('', text)

def remove_hashtags(text):
    return re.sub(r"#\w+", "", text)

# 4. Đếm số từ sau khi loại emoji và hashtag
def count_words_cleaned(text):
    cleaned = remove_emojis(remove_hashtags(text))
    words = cleaned.strip().split()
    return len(words)

In [127]:
# Chia thành 5 khung giờ
def get_time_slot(hour):
    if 0 <= hour < 6:
        return "early_morning"
    elif 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 18:
        return "afternoon"
    elif 18 <= hour < 22:
        return "evening"
    else:
        return "late_night"

In [124]:
# Tạo danh sách username
list_kol = pd.read_excel('List_KOL_final.xlsx', sheet_name='Sheet1')
# Lọc DataFrame để chỉ lấy các hàng mà cột 'erdeleted_kolror' không phải là null
kol_with_errors = list_kol[list_kol['deleted_kol'].isna()]
kol_usernames = kol_with_errors["name"].tolist()

In [128]:
csv_files = [name + '.csv' for name in kol_usernames]

In [141]:
len(csv_files)

82

In [ ]:
# === Thư mục
input_folder = './kol_data_with_istrend/'
output_folder = './kol_data_csv_final'
os.makedirs(output_folder, exist_ok=True)

# === Duyệt và xử lý từng file
for file_name in tqdm(csv_files, desc="Đang xử lý các file CSV"):
    input_path = os.path.join(input_folder, file_name)
    output_path = os.path.join(output_folder, file_name)

    try:
        df = pd.read_csv(input_path, parse_dates=["createTime"])

        # 1. Thuộc tính từ caption
        df['text'] = df['text'].fillna('').astype(str)
        df['emoji_count'] = df['text'].apply(count_emojis)
        df['hashtag_count'] = df['text'].apply(count_hashtags)
        df['word_count_no_emoji_hashtag'] = df['text'].apply(count_words_cleaned)

        # 2. Khung thời gian
        df['time_slot'] = df['createTime'].dt.hour.map(get_time_slot)

        for slot in ["early_morning", "morning", "afternoon", "evening", "late_night"]:
            df[slot] = (df['time_slot'] == slot).astype(int)

        df.drop(columns=["time_slot"], inplace=True)

        # 3. Cuối tuần
        df['is_weekend'] = (df['createTime'].dt.dayofweek >= 5).astype(int)

        # 4. Ghi file mới
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✅ Đã xử lý: {file_name}")

    except Exception as e:
        print(f"❌ Lỗi file {file_name}: {str(e)}")

---

# Phân tích Follower+

In [ ]:
folder_path = './Data_level_II_hoang' 
csv_files_follower = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

print(csv_files_follower)

['hnhu2804.csv', 'dramakinglndx.csv', 'im.lunadao_official.csv', 'biveve131.csv', 'thanhyvo.csv', 'khietnhinhi.csv', 'hoangminhngoc21.csv', 'msquynhthie.csv', 'goc.cua.ru.csv', 'beautytips4.0.csv', 'chouchinchan.csv', 'ph.hoanganhh.csv', 'lethikhanhhuyen2004.csv', 'bychloenguyen.csv', 'nhatkycuameichan.csv', 'rosermae.csv', 'drkhonghanhnguyen.csv', 'emma1010_official.csv', 'bylinhtruong.csv', 'blingbabi_.csv', 'lihtrag.0903.csv', 'mai.trinh.h.csv', 'thudannguyen99.csv', 'im.lukedo.csv', 'vznganmin.csv', 'rinavaseoul2003.csv', 'kimchungphan20.csv', 'halinhofficial.csv', 'yendan7.csv', 'minzy.in.kr.csv', 'hannaholala.csv', 'chaubui.csv', 'tnhadinh.csv', 'lebong95.csv', 'luongthuchien6868.csv', 'tomskincare.csv', 'hanhxuka2804.csv', 'linhtruong.ld.csv', 'han.hang18.csv', 'gigigithebabe.csv', 'ngoctrinh89.csv', 'michanne1011.csv', 'trixiebaeee.csv', 'trinhpham2222.csv', '_inlil.csv', 'phuonglytran.csv', 'eb06thg9.csv', 'tran_lam18.csv', 'dieplamanh.csv', 'lepthelittlekid.csv', 'actress.lan

In [143]:
len(csv_files_follower)

53

In [145]:
len(csv_files)

82

In [146]:
set_follower = set(csv_files_follower)
set_all      = set(csv_files)

# 1️⃣ Các file trùng nhau
common_files = set_follower & set_all
print(f"Số file trùng nhau: {len(common_files)}")
print("Các file trùng nhau:", common_files)

# 2️⃣ Các file có trong follower nhưng không có trong thư mục
only_in_follower = set_follower - set_all
print(f"\nCó trong csv_files_follower nhưng không trong csv_files ({len(only_in_follower)}):")
print(only_in_follower)

# 3️⃣ Các file có trong thư mục nhưng không có trong follower
only_in_csv = set_all - set_follower
print(f"\nCó trong csv_files nhưng không trong csv_files_follower ({len(only_in_csv)}):")
print(only_in_csv)

Số file trùng nhau: 53
Các file trùng nhau: {'im.lunadao_official.csv', 'lihtrag.0903.csv', 'lethikhanhhuyen2004.csv', 'dieplamanh.csv', 'linhtruong.ld.csv', 'khietnhinhi.csv', 'trinhpham2222.csv', 'rinavaseoul2003.csv', 'chaubui.csv', 'eb06thg9.csv', 'kimchungphan20.csv', 'mai.trinh.h.csv', 'beautytips4.0.csv', 'tomskincare.csv', 'luongthuchien6868.csv', 'thudannguyen99.csv', 'bylinhtruong.csv', 'ph.hoanganhh.csv', 'biveve131.csv', 'tnhadinh.csv', 'lebong95.csv', 'imneyy.csv', 'hnhu2804.csv', 'minzy.in.kr.csv', '_inlil.csv', 'yendan7.csv', 'trixiebaeee.csv', 'hanhxuka2804.csv', 'gigigithebabe.csv', 'nhatkycuameichan.csv', 'dramakinglndx.csv', 'im.lukedo.csv', 'vznganmin.csv', 'ngoctrinh89.csv', 'emma1010_official.csv', 'han.hang18.csv', 'hannaholala.csv', 'bychloenguyen.csv', 'michanne1011.csv', 'chouchinchan.csv', 'tran_lam18.csv', 'goc.cua.ru.csv', 'actress.lanhuong.csv', 'halinhofficial.csv', 'phuonglytran.csv', 'hoangminhngoc21.csv', 'lepthelittlekid.csv', 'blingbabi_.csv', 'msquy

\- Chứng tỏ rằng chỉ có 53/82 follower là có dữ liệu về `Followers+`

In [152]:
folder_path = './Data_level_II_hoang'
total_zero = 0

for filename in csv_files_follower:
    file_path = os.path.join(folder_path, filename)

    try:
        df = pd.read_csv(file_path)

        # Xử lý tên cột: loại bỏ khoảng trắng, kiểm tra kỹ tên cột
        df.columns = df.columns.str.strip()

        if 'Followers+' in df.columns:
            num_zero = (df['Followers+'] == 0).sum()
            total_zero += num_zero
            print(f"{filename}: {num_zero}")
        else:
            print(f"{filename}: ⚠️ Không có cột 'Followers+' (tên thật: {list(df.columns)})")

    except Exception as e:
        print(f"{filename}: ❌ Lỗi khi đọc file — {e}")

hnhu2804.csv: 9
dramakinglndx.csv: 0
im.lunadao_official.csv: 12
biveve131.csv: 1
thanhyvo.csv: 1
khietnhinhi.csv: 4
hoangminhngoc21.csv: 12
msquynhthie.csv: 11
goc.cua.ru.csv: 2
beautytips4.0.csv: 4
chouchinchan.csv: 13
ph.hoanganhh.csv: 10
lethikhanhhuyen2004.csv: 2
bychloenguyen.csv: 4
nhatkycuameichan.csv: 0
rosermae.csv: 8
drkhonghanhnguyen.csv: 15
emma1010_official.csv: 4
bylinhtruong.csv: 5
blingbabi_.csv: 11
lihtrag.0903.csv: 4
mai.trinh.h.csv: 16
thudannguyen99.csv: 3
im.lukedo.csv: 7
vznganmin.csv: 11
rinavaseoul2003.csv: 7
kimchungphan20.csv: 2
halinhofficial.csv: 0
yendan7.csv: 6
minzy.in.kr.csv: 4
hannaholala.csv: 1
chaubui.csv: 7
tnhadinh.csv: 4
lebong95.csv: 1
luongthuchien6868.csv: 2
tomskincare.csv: 2
hanhxuka2804.csv: 14
linhtruong.ld.csv: 17
han.hang18.csv: 17
gigigithebabe.csv: 0
ngoctrinh89.csv: 2
michanne1011.csv: 15
trixiebaeee.csv: 15
trinhpham2222.csv: 16
_inlil.csv: 10
phuonglytran.csv: 8
eb06thg9.csv: 1
tran_lam18.csv: 1
dieplamanh.csv: 12
lepthelittlekid.csv

In [153]:
print(total_zero)

379


\- ta thấy rằng nếu 29 kol mà không có follower mà nó có đủ cả từ tháng 1/2024 đến tháng 5/2025 thì ta sẽ phải điền thủ công 872 tháng nữa.

In [ ]:
# Bây giờ sẽ xây dừng data level 2, rồi sau đó sẽ chia đều file ra để điền thủ công
# và giới thiệu cho Hoàng một extension trên vscode để điền thủ công và lưu lại luôn.

# Kiểm tra xem là ở các KOL thì họ có đủ tháng để chia thành 6 input và 3 output không.

# Sau đó xây dựng data level 3 (6 tháng thì sẽ tạo ra cột isOutlier với khoảng thời gian xét 3 tháng. Ngoài ra tạo thêm một thuộc tính số bài nhặc sử
# dụng độc nhất trên cả 6 tháng ).



In [154]:
len(csv_files)

82

In [156]:
folder_path = './kol_data_csv_final'  # Đổi nếu cần

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path)

        if 'createTime' not in df.columns:
            print(f"{file_name}: ⚠️ Không có cột 'createTime'")
            continue

        df['createTime'] = pd.to_datetime(df['createTime'], errors='coerce')
        df = df.dropna(subset=['createTime'])

        # Trích Năm và Tháng
        df['year'] = df['createTime'].dt.year
        df['month'] = df['createTime'].dt.month

        # Đếm số dòng theo (năm, tháng)
        grouped = df.groupby(['year', 'month']).size().reset_index(name='count')

        print(f"\n📂 File: {file_name}")
        for _, row in grouped.iterrows():
            print(f"  - Năm {int(row['year'])}, Tháng {int(row['month'])}: {row['count']} dòng")
        print(f"  Tổng số tháng có dữ liệu: {grouped.shape[0]}")

    except Exception as e:
        print(f"{file_name}: ❌ Lỗi khi xử lý - {e}")


📂 File: halinhofficial.csv
  - Năm 2024, Tháng 1: 9 dòng
  - Năm 2024, Tháng 2: 9 dòng
  - Năm 2024, Tháng 3: 9 dòng
  - Năm 2024, Tháng 4: 12 dòng
  - Năm 2024, Tháng 5: 10 dòng
  - Năm 2024, Tháng 6: 13 dòng
  - Năm 2024, Tháng 7: 9 dòng
  - Năm 2024, Tháng 8: 16 dòng
  - Năm 2024, Tháng 9: 14 dòng
  - Năm 2024, Tháng 10: 12 dòng
  - Năm 2024, Tháng 11: 16 dòng
  - Năm 2024, Tháng 12: 16 dòng
  - Năm 2025, Tháng 1: 7 dòng
  - Năm 2025, Tháng 2: 8 dòng
  - Năm 2025, Tháng 3: 17 dòng
  - Năm 2025, Tháng 4: 9 dòng
  - Năm 2025, Tháng 5: 15 dòng
  - Năm 2025, Tháng 6: 5 dòng
  Tổng số tháng có dữ liệu: 18

📂 File: hannaholala.csv
  - Năm 2024, Tháng 1: 20 dòng
  - Năm 2024, Tháng 2: 21 dòng
  - Năm 2024, Tháng 3: 16 dòng
  - Năm 2024, Tháng 4: 22 dòng
  - Năm 2024, Tháng 5: 29 dòng
  - Năm 2024, Tháng 6: 18 dòng
  - Năm 2024, Tháng 7: 20 dòng
  - Năm 2024, Tháng 8: 20 dòng
  - Năm 2024, Tháng 9: 24 dòng
  - Năm 2024, Tháng 10: 25 dòng
  - Năm 2024, Tháng 11: 24 dòng
  - Năm 2024, Tháng 

In [167]:
folder_path = './kol_data_csv_final'  # Đường dẫn thư mục chứa CSV
valid_csv_files = []
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path)

        if 'createTime' not in df.columns:
            print(f"{file_name}: ⚠️ Không có cột 'createTime'")
            continue

        df['createTime'] = pd.to_datetime(df['createTime'], errors='coerce')
        df = df.dropna(subset=['createTime'])

        # Tạo cột năm và tháng
        df['year'] = df['createTime'].dt.year
        df['month'] = df['createTime'].dt.month

        # ❌ Loại bỏ riêng tháng 6 của năm 2025
        df = df[~((df['year'] == 2025) & (df['month'] == 6))]

        # Tạo 'year-month' dưới dạng Period
        df['year_month'] = df['createTime'].dt.to_period('M')

        # Đếm số tháng duy nhất
        unique_months = df['year_month'].dropna().unique()
        num_months = len(unique_months)
        
        # ✅ Lưu file nếu đủ 10 tháng trở lên
        if num_months >= 10:
            valid_csv_files.append(file_name)

        if num_months < 10:
            print(f"\n❗ File: {file_name}")
            print(f"  - Số tháng có dữ liệu (không tính tháng 6/2025): {num_months}")
            print(f"  - Danh sách tháng: {[str(m) for m in sorted(unique_months)]}")

    except Exception as e:
        print(f"{file_name}: ❌ Lỗi khi xử lý - {e}")


❗ File: nguyenhuemakeup1996.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 8
  - Danh sách tháng: ['2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05']

❗ File: hnhu2804.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 8
  - Danh sách tháng: ['2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05']

❗ File: thanhhuongord.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 2
  - Danh sách tháng: ['2025-04', '2025-05']

❗ File: luongthuchien6868.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 6
  - Danh sách tháng: ['2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05']

❗ File: huynhanhthu_cosmetic.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 5
  - Danh sách tháng: ['2024-04', '2024-09', '2024-10', '2025-01', '2025-05']

❗ File: annie.rose697.csv
  - Số tháng có dữ liệu (không tính tháng 6/2025): 2
  - Danh sách tháng: ['2025-04', '2025-05']

❗ File: hienlamdep90.csv
  - Số th

In [182]:
len(valid_csv_files)

75

In [170]:
input_folder = './kol_data_csv_final'
follower_folder = './Data_level_II_hoang'
output_folder = './kol_follower_summary'

for file_name in valid_csv_files:
    input_path = os.path.join(input_folder, file_name)
    follower_path = os.path.join(follower_folder, file_name)

    try:
        if os.path.exists(follower_path):
            # 📂 Nếu có file follower
            df_follower = pd.read_csv(follower_path)

            # Tách tháng/năm từ 'Month_Year' rồi ép kiểu
            df_follower[['month', 'year']] = df_follower['Month_Year'].str.extract(r'(\d{1,2})[/-](\d{4})').astype(int)

            # ❌ Loại bỏ tháng 6 năm 2025
            df_follower = df_follower[~((df_follower['year'] == 2025) & (df_follower['month'] == 6))]

            # Tạo dataframe xuất
            df_final = df_follower[['year', 'month', 'Followers+']].rename(columns={'Followers+': 'followers'})
            df_final = df_final.sort_values(['year', 'month'])

        else:
            # 📂 Nếu KHÔNG có file follower
            df = pd.read_csv(input_path, parse_dates=['createTime'])  # ✅ ép kiểu luôn

            if 'createTime' not in df.columns:
                print(f"{file_name}: ⚠️ Không có cột 'createTime'")
                continue

            df = df.dropna(subset=['createTime'])

            # Trích tháng và năm
            df['year'] = df['createTime'].dt.year
            df['month'] = df['createTime'].dt.month

            # ❌ Loại bỏ tháng 6 năm 2025
            df = df[~((df['year'] == 2025) & (df['month'] == 6))]

            # Đếm theo tháng
            month_summary = df.groupby(['year', 'month']).size().reset_index(name='post_count')
            month_summary['followers'] = 0  # follower mặc định = 0

            df_final = month_summary[['year', 'month', 'followers']].sort_values(['year', 'month'])

        # 💾 Lưu kết quả
        output_path = os.path.join(output_folder, file_name)
        df_final.to_csv(output_path, index=False)
        print(f"✅ Đã xử lý: {file_name}")

    except Exception as e:
        print(f"❌ Lỗi với file {file_name}: {e}")

✅ Đã xử lý: halinhofficial.csv
✅ Đã xử lý: hannaholala.csv
✅ Đã xử lý: goc.cua.ru.csv
✅ Đã xử lý: mai.trinh.h.csv
✅ Đã xử lý: tomskincare.csv
✅ Đã xử lý: rosermae.csv
✅ Đã xử lý: chouchinchan.csv
✅ Đã xử lý: trinhpham2222.csv
✅ Đã xử lý: bylinhtruong.csv
✅ Đã xử lý: maivantrang_.csv
✅ Đã xử lý: linhtruong.ld.csv
✅ Đã xử lý: ngoctrinh89.csv
✅ Đã xử lý: dramakinglndx.csv
✅ Đã xử lý: msquynhthie.csv
✅ Đã xử lý: truc.dao.csv
✅ Đã xử lý: dieplamanh.csv
✅ Đã xử lý: lehatruc.daily.csv
✅ Đã xử lý: bychloenguyen.csv
✅ Đã xử lý: im.lunadao_official.csv
✅ Đã xử lý: hoangminhngoc21.csv
✅ Đã xử lý: thaonhileofficial.csv
✅ Đã xử lý: actress.lanhuong.csv
✅ Đã xử lý: blingbabi_.csv
✅ Đã xử lý: hwitch99.csv
✅ Đã xử lý: emmi.hoang.csv
✅ Đã xử lý: thaovann_0602.csv
✅ Đã xử lý: chaubui.csv
✅ Đã xử lý: nhatkycuameichan.csv
✅ Đã xử lý: tnhadinh.csv
✅ Đã xử lý: thudannguyen99.csv
✅ Đã xử lý: thanhyvo.csv
✅ Đã xử lý: ph.hoanganhh.csv
✅ Đã xử lý: myhuyen_channel.csv
✅ Đã xử lý: vznganmin.csv
✅ Đã xử lý: rinava

## Chia file ra để điền thủ công

In [171]:
folder_path = './kol_follower_summary'  # Thư mục chứa các file đã chuẩn hoá (có cột year, month, followers)

follower_zero_counts = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path)
            count_zero = (df['followers'] == 0).sum()
            follower_zero_counts.append((file, count_zero))
        except Exception as e:
            print(f"Lỗi với file {file}: {e}")

In [172]:
# Sắp xếp theo số dòng có follower == 0
sorted_files = sorted(follower_zero_counts, key=lambda x: x[1], reverse=True)

In [173]:
group_1 = []
group_2 = []
sum_1 = 0
sum_2 = 0

for file, count in sorted_files:
    if sum_1 <= sum_2:
        group_1.append(file)
        sum_1 += count
    else:
        group_2.append(file)
        sum_2 += count

In [174]:
with open("group_1.txt", "w") as f:
    for file in group_1:
        f.write(file + "\n")

with open("group_2.txt", "w") as f:
    for file in group_2:
        f.write(file + "\n")

In [180]:
# Bước 4: Sao chép file vào thư mục tương ứng
output_dir_1 = './to_fill_group_1'
output_dir_2 = './to_fill_group_2'

# Tạo thư mục đích nếu chưa tồn tại
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)
for file in group_1:
    shutil.copy(os.path.join(folder_path, file), os.path.join(output_dir_1, file))

for file in group_2:
    shutil.copy(os.path.join(folder_path, file), os.path.join(output_dir_2, file))

print(f"✅ Đã chia {len(follower_zero_counts)} file thành:")
print(f"  📂 {output_dir_1}: {len(group_1)} file (tổng dòng cần điền: {sum_1})")
print(f"  📂 {output_dir_2}: {len(group_2)} file (tổng dòng cần điền: {sum_2})")

✅ Đã chia 75 file thành:
  📂 ./to_fill_group_1: 42 file (tổng dòng cần điền: 356)
  📂 ./to_fill_group_2: 33 file (tổng dòng cần điền: 356)


---
# Tạo Dataset level 2

In [181]:
test_df = pd.read_csv('./kol_data_csv_final/_inlil.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   text                         271 non-null    object
 1   createTime                   275 non-null    object
 2   createTimeISO                275 non-null    object
 3   isAd                         275 non-null    bool  
 4   profileUrl                   275 non-null    object
 5   musicName                    273 non-null    object
 6   musicId                      275 non-null    int64 
 7   webVideoUrl                  275 non-null    object
 8   duration                     275 non-null    int64 
 9   diggCount                    275 non-null    int64 
 10  shareCount                   275 non-null    int64 
 11  playCount                    275 non-null    int64 
 12  collectCount                 275 non-null    int64 
 13  commentCount                 275 no

In [ ]:
# === CẤU HÌNH THƯ MỤC ===
INPUT_FOLDER = "./kol_data_csv_final"
OUTPUT_FOLDER = "./kol_data_csv_6_month_backup"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === CỘT CẦN TỔNG HỢP THEO PR / NON-PR ===
AGG_COLS = [
    "emoji_count", "hashtag_count", "word_count_no_emoji_hashtag",
    "early_morning", "morning", "afternoon", "evening", "late_night", "is_weekend",
    "is_trending", "duration", "diggCount", "shareCount", "collectCount",
    "commentCount", "playCount", "isSlideshow", "isSponsored"
]

# === KHỞI TẠO SLIDING WINDOW CÓ CẢ TARGET ===
def generate_windows():
    windows = []
    for i in range(8):  # Điều chỉnh số lượng window tùy theo dữ liệu
        input_start = Period("2024-01") + i
        input_end = input_start + 5  # 6 tháng input
        target_start = input_end + 2  # +1 tháng buffer +1 nữa để bắt đầu target
        target_end = target_start + 2  # 3 tháng target

        windows.append({
            "input_start": str(input_start),
            "input_end": str(input_end),
            "target_start": str(target_start),
            "target_end": str(target_end)
        })
    return windows

WINDOWS = generate_windows()

def count_e_outlier(df_group):
    if df_group.empty:
        return 0
    # Tính tổng engagement từng video
    engagement = (
        df_group["diggCount"] + df_group["shareCount"] +
        df_group["collectCount"] + df_group["commentCount"]
    )
    Q1 = engagement.quantile(0.25)
    Q3 = engagement.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    return (engagement > upper_bound).sum()

# === XỬ LÝ TỪNG FILE ===
def process_file(file_path):
    df = pd.read_csv(file_path, parse_dates=["createTime"])
    df["year_month"] = df["createTime"].dt.to_period("M")

    available_months = df["year_month"].unique()
    results = []

    for win in WINDOWS:
        input_start = pd.Period(win["input_start"])
        input_end = pd.Period(win["input_end"])
        target_start = pd.Period(win["target_start"])
        target_end = pd.Period(win["target_end"])

        # Các tháng cần kiểm tra
        input_months = [input_start + i for i in range(6)]
        target_months = [target_start + i for i in range(3)]

        # Nếu không đủ tháng input hoặc target thì bỏ qua
        if not all(m in available_months for m in input_months + target_months):
            continue

        # === PHẦN INPUT ===
        input_df = df[df["year_month"].isin(input_months)]
        pr_df = input_df[input_df["isAd"] == True]
        nonpr_df = input_df[input_df["isAd"] == False]

        row = {
            "kol_username": df["kol_username"].iloc[0],
            "month_start": int(input_start.month),
            "year_start": int(input_start.year),
            "month_end": int(input_end.month),
            "year_end": int(input_end.year),
            "total_posts_pr": len(pr_df),
            "total_posts_nonpr": len(nonpr_df),
            "unique_music_pr": pr_df["musicId"].nunique(),
            "unique_music_nonpr": nonpr_df["musicId"].nunique(),
            "ttSeller": df["ttSeller"].iloc[0],
            "commerceUser": df["commerceUser"].iloc[0],
        }

        for col in AGG_COLS:
            row[f"{col}_pr"] = pr_df[col].sum()
            row[f"{col}_nonpr"] = nonpr_df[col].sum()

        # Đếm số video vượt outlier
        row["e_outlier_pr"] = count_e_outlier(pr_df)
        row["e_outlier_nonpr"] = count_e_outlier(nonpr_df)

        # === TÍNH TỔNG FOLLOWER TRONG 6 THÁNG INPUT ===
        kol_name = df["kol_username"].iloc[0]
        follower_path = os.path.join("./kol_follower_summary", f"{kol_name}.csv")

        if os.path.exists(follower_path):
            df_f = pd.read_csv(follower_path)
            # Tạo cột period để dễ so sánh
            df_f["period"] = df_f["year"].astype(str) + "-" + df_f["month"].astype(str).str.zfill(2)
            df_f["period"] = pd.PeriodIndex(df_f["period"], freq="M")
            # Lọc theo 6 tháng input
            input_follower_df = df_f[df_f["period"].isin(input_months)]
            total_followers = input_follower_df["followers"].sum()
        else:
            total_followers = 0  # hoặc np.nan nếu muốn phân biệt

        row["follower_change_in_input_window"] = total_followers
        

        # === PHẦN TARGET ===
        target_pr_df = df[
            (df["year_month"].isin(target_months)) & (df["isAd"] == True)
        ]

        # Nếu không có bài PR nào trong 3 tháng target thì bỏ qua
        if target_pr_df.empty:
            continue  # Bỏ qua luôn cửa sổ này

        # Nếu có bài PR nhưng playCount = 0 thì target là 0
        total_engagement = (
            target_pr_df["diggCount"].sum()
            + target_pr_df["shareCount"].sum()
            + target_pr_df["collectCount"].sum()
            + target_pr_df["commentCount"].sum()
        )
        total_play = target_pr_df["playCount"].sum()

        if total_play > 0:
            engagement_rate = total_engagement / total_play
        else:
            engagement_rate = 0  # vẫn giữ lại, không bỏ qua

        # Cuối cùng gán target
        row["EngagementRateOnPRPost_target"] = engagement_rate

        results.append(row)

    return pd.DataFrame(results)

# === XỬ LÝ TOÀN BỘ FILE TRONG THƯ MỤC ===
all_dfs = []
for file in valid_csv_files:
    filepath = os.path.join(INPUT_FOLDER, file)
    result_df = process_file(filepath)

    if not result_df.empty:
        out_path = os.path.join(OUTPUT_FOLDER, file)
        result_df.to_csv(out_path, index=False)
        all_dfs.append(result_df)

# === GỘP FILE TỔNG ===
FINAL_DATA_FOLDER = './kol_window_slidding'
os.makedirs(FINAL_DATA_FOLDER, exist_ok=True)
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(os.path.join(FINAL_DATA_FOLDER, "all_kols_combined.csv"), index=False)


In [193]:
len(valid_csv_files)

75

In [ ]:
# hàm thêm các thuộc tính về tỉ lệ
def add_selected_ratio_features(file_path):
    df = pd.read_csv(file_path)
    if df.empty:
        return

    safe_div = lambda num, denom: np.where(denom == 0, 0, num / denom)

    # === Tỷ lệ thời gian đăng bài PR ===
    for time in ["early_morning", "morning", "afternoon", "evening", "late_night"]:
        df[f"pr_ratio_{time}"] = safe_div(df[f"{time}_pr"], df["total_posts_pr"])

    # === Tỷ lệ nội dung slideshow, sponsored trên PR ===
    df["pr_ratio_slideshow"] = safe_div(df["isSlideshow_pr"], df["total_posts_pr"])
    df["pr_ratio_sponsored"] = safe_div(df["isSponsored_pr"], df["total_posts_pr"])

    # === Tỷ lệ trending và weekend ===
    df["pr_ratio_trending"] = safe_div(df["is_trending_pr"], df["total_posts_pr"])
    df["pr_ratio_weekend"] = safe_div(df["is_weekend_pr"], df["total_posts_pr"])

    # === Tỷ lệ bài PR / tổng số bài ===
    df["pr_post_ratio"] = safe_div(
        df["total_posts_pr"], df["total_posts_pr"] + df["total_posts_nonpr"]
    )

    # === Tỷ lệ engagement trên bài PR ===
    df["pr_engagement_rate_input"] = safe_div(
        df["diggCount_pr"] + df["shareCount_pr"] + df["collectCount_pr"] + df["commentCount_pr"],
        df["playCount_pr"]
    )

    # === Tỷ lệ bài PR là outlier ===
    df["pr_ratio_e_outlier"] = safe_div(df["e_outlier_pr"], df["total_posts_pr"])

    # Ghi đè lại file
    df.to_csv(file_path, index=False)

for csv_file in glob.glob(os.path.join(OUTPUT_FOLDER, "*.csv")):
    if not csv_file.endswith("all_kols_combined.csv"):
        add_selected_ratio_features(csv_file)

# Cập nhật file tổng sau khi có các feature mới
updated_dfs = []
for file in valid_csv_files:
    updated_path = os.path.join(OUTPUT_FOLDER, file)
    if os.path.exists(updated_path):
        updated_dfs.append(pd.read_csv(updated_path))


In [ ]:
# Công việc ngày mai
# Chạy lại để tạo list csv cho thư mục 6 months. Sẽ xoá những dòng có target = 0.
# Thêm vào 2 thuộc tính mới trong hàm là thuộc tính tổng follower biến động qua từng tháng ( tức là số follower mới tăng hoặc giảm sau 6 tháng đó)
# Thêm thuộc tính is_outlier tức đếm xem là có bao nhiêu bài đăng trong 6 tháng là nằm trong outlier ( tôi không sét trên 3 tháng và cộng lại chia đôi vì với 1 nhóm 3 tháng thì số bài đăng sẽ khác nhau )

In [203]:
check = pd.read_csv('./window_slidding_data/all_kols_combined.csv')
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 63 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   kol_username                       463 non-null    object 
 1   month_start                        463 non-null    int64  
 2   year_start                         463 non-null    int64  
 3   month_end                          463 non-null    int64  
 4   year_end                           463 non-null    int64  
 5   total_posts_pr                     463 non-null    int64  
 6   total_posts_nonpr                  463 non-null    int64  
 7   unique_music_pr                    463 non-null    int64  
 8   unique_music_nonpr                 463 non-null    int64  
 9   ttSeller                           463 non-null    bool   
 10  commerceUser                       463 non-null    bool   
 11  emoji_count_pr                     463 non-null    int64  

In [200]:
check['kol_username'].nunique()

69

In [202]:
# === Đường dẫn chứa file CSV đã có ===
REARRANGE_FOLDER = './kol_data_csv_6_month'

# === Đọc lại file CSV bất kỳ để lấy danh sách cột mẫu ===
sample_file = glob.glob(os.path.join(REARRANGE_FOLDER, "*.csv"))[0]
df = pd.read_csv(sample_file)

# === Tìm các nhóm biến ===
raw_cols = list(df.columns)

# 1. Cột không phải tỷ lệ
base_cols = [col for col in raw_cols if not col.startswith("pr_ratio_") and col not in [
    "pr_engagement_rate_input", "pr_post_ratio", "pr_ratio_e_outlier", "EngagementRateOnPRPost_target"
]]

# 2. Cột tỷ lệ theo thời gian (gắn ngay sau các biến thời gian PR)
time_slots = ["early_morning", "morning", "afternoon", "evening", "late_night"]
time_ratio_cols = []
for slot in time_slots:
    base_cols.insert(base_cols.index(f"{slot}_pr") + 1, f"pr_ratio_{slot}")
    time_ratio_cols.append(f"pr_ratio_{slot}")

# 3. Các tỷ lệ khác gắn gần phần tương ứng
insert_after = {
    "isSlideshow_pr": "pr_ratio_slideshow",
    "isSponsored_pr": "pr_ratio_sponsored",
    "is_trending_pr": "pr_ratio_trending",
    "is_weekend_pr": "pr_ratio_weekend",
    "e_outlier_pr": "pr_ratio_e_outlier"
}
for key_col, ratio_col in insert_after.items():
    base_cols.insert(base_cols.index(key_col) + 1, ratio_col)

# 4. Các tỷ lệ tổng hợp khác
extra_ratios = ["pr_engagement_rate_input", "pr_post_ratio"]
for col in extra_ratios:
    base_cols.append(col)

# 5. Cuối cùng là cột target
base_cols.append("EngagementRateOnPRPost_target")

# === Hàm áp dụng sắp xếp lại cột và ghi đè ===
def rearrange_columns(file_path):
    df = pd.read_csv(file_path)
    df = df[base_cols]
    df.to_csv(file_path, index=False)

# === Áp dụng toàn bộ file trong thư mục ===
for file_path in glob.glob(os.path.join(REARRANGE_FOLDER, "*.csv")):
    if file_path.endswith("all_kols_combined.csv"):
        continue
    rearrange_columns(file_path)

# === Sắp xếp lại file tổng sau khi các file con đã cập nhật ===
updated_dfs = []
for file_path in glob.glob(os.path.join(REARRANGE_FOLDER, "*.csv")):
    if file_path.endswith("all_kols_combined.csv"):
        continue
    updated_dfs.append(pd.read_csv(file_path))

if updated_dfs:
    final_df = pd.concat(updated_dfs, ignore_index=True)
    final_df = final_df[base_cols]
    final_df.to_csv(os.path.join('./window_slidding_data', "all_kols_combined.csv"), index=False)


# Gộp thành dataset cho kiểm định

In [204]:
# === Thư mục chứa các file CSV đã xử lý theo từng KOL ===
SEGMENT_FOLDER = './kol_data_csv_6_month'

# === Thư mục đích để chứa file kiểm định ===
VALIDATION_FOLDER = './kol_data_for_validation'
os.makedirs(VALIDATION_FOLDER, exist_ok=True)

# === Danh sách các dòng cần lấy ra từ mỗi file ===
validation_rows = []

for file_path in glob.glob(os.path.join(SEGMENT_FOLDER, "*.csv")):
    if file_path.endswith("all_kols_combined.csv"):
        continue  # bỏ qua file tổng nếu có

    df = pd.read_csv(file_path)

    # Lọc điều kiện theo yêu cầu
    filtered = df[
        (df["month_start"] == 8) &
        (df["year_start"] == 2024) &
        (df["month_end"] == 1) &
        (df["year_end"] == 2025)
    ]

    if not filtered.empty:
        validation_rows.append(filtered)

# Gộp lại thành một DataFrame
if validation_rows:
    df_validation = pd.concat(validation_rows, ignore_index=True)
    output_path = os.path.join(VALIDATION_FOLDER, "validation_data.csv")
    df_validation.to_csv(output_path, index=False)
    print(f"✅ File kiểm định đã được tạo tại: {output_path}")
else:
    print("⚠️ Không tìm thấy dòng nào thỏa mãn điều kiện.")

✅ File kiểm định đã được tạo tại: ./kol_data_for_validation/validation_data.csv


In [205]:
check = pd.read_csv('./kol_data_for_validation/validation_data.csv')
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 63 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   kol_username                       61 non-null     object 
 1   month_start                        61 non-null     int64  
 2   year_start                         61 non-null     int64  
 3   month_end                          61 non-null     int64  
 4   year_end                           61 non-null     int64  
 5   total_posts_pr                     61 non-null     int64  
 6   total_posts_nonpr                  61 non-null     int64  
 7   unique_music_pr                    61 non-null     int64  
 8   unique_music_nonpr                 61 non-null     int64  
 9   ttSeller                           61 non-null     bool   
 10  commerceUser                       61 non-null     bool   
 11  emoji_count_pr                     61 non-null     int64  
 

In [206]:
df_fl = pd.read_csv('data_fl.csv')

In [209]:
df_fl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  57 non-null     int64 
 1   name_kol    57 non-null     object
 2   female      56 non-null     object
 3   male        56 non-null     object
 4   vietnam     57 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.4+ KB


# Data for Training model

In [210]:
# === Đọc dữ liệu follower demographics ===
df_fl = pd.read_csv("data_fl.csv")

# === Chuyển '64%' -> 64.0 ===
for col in ["female", "male"]:
    df_fl[col] = df_fl[col].str.replace('%', '', regex=False).astype(float)

# === Đổi tên cột để rõ ràng hơn ===
df_fl.rename(columns={
    "female": "female_percent",
    "male": "male_percent",
    "vietnam": "vietnam_follower"
}, inplace=True)

# === Hàm để merge demographic info vào 1 file cụ thể ===
def merge_fl_info(target_path, keep_cols):
    if not os.path.exists(target_path):
        print(f"File không tồn tại: {target_path}")
        return

    df_target = pd.read_csv(target_path)

    # Chỉ lấy các cột cần thiết trong demographics
    df_merge_part = df_fl[["name_kol"] + keep_cols]

    df_merged = df_target.merge(df_merge_part, how="left", left_on="kol_username", right_on="name_kol")
    df_merged.drop(columns=["name_kol"], inplace=True)

    # Đưa EngagementRateOnPRPost_target xuống cuối cùng (nếu có)
    target_col = "EngagementRateOnPRPost_target"
    if target_col in df_merged.columns:
        cols = [col for col in df_merged.columns if col != target_col] + [target_col]
        df_merged = df_merged[cols]

    df_merged.to_csv(target_path, index=False)
    print(f"Đã cập nhật: {target_path}")

# === Ghép vào file chính (chỉ lấy female + vietnam) ===
merge_fl_info(
    "./window_slidding_data/all_kols_combined.csv",
    keep_cols=["female_percent", "vietnam_follower"]
)

# === Ghép vào file kiểm định (lấy đầy đủ 3 cột) ===
merge_fl_info(
    "./kol_data_for_validation/validation_data.csv",
    keep_cols=["female_percent", "male_percent", "vietnam_follower"]
)


Đã cập nhật: ./window_slidding_data/all_kols_combined.csv
Đã cập nhật: ./kol_data_for_validation/validation_data.csv


In [216]:
check = pd.read_csv('./window_slidding_data/all_kols_combined.csv')
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 65 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   kol_username                       463 non-null    object 
 1   month_start                        463 non-null    int64  
 2   year_start                         463 non-null    int64  
 3   month_end                          463 non-null    int64  
 4   year_end                           463 non-null    int64  
 5   total_posts_pr                     463 non-null    int64  
 6   total_posts_nonpr                  463 non-null    int64  
 7   unique_music_pr                    463 non-null    int64  
 8   unique_music_nonpr                 463 non-null    int64  
 9   ttSeller                           463 non-null    bool   
 10  commerceUser                       463 non-null    bool   
 11  emoji_count_pr                     463 non-null    int64  

In [213]:
check['vietnam_follower']

0      90%
1      90%
2      90%
3      90%
4      90%
      ... 
458    91%
459    91%
460    91%
461    91%
462    91%
Name: vietnam_follower, Length: 463, dtype: object

In [214]:
df_val = pd.read_csv('./kol_data_for_validation/validation_data.csv')

In [215]:
df_val.columns

Index(['kol_username', 'month_start', 'year_start', 'month_end', 'year_end',
       'total_posts_pr', 'total_posts_nonpr', 'unique_music_pr',
       'unique_music_nonpr', 'ttSeller', 'commerceUser', 'emoji_count_pr',
       'emoji_count_nonpr', 'hashtag_count_pr', 'hashtag_count_nonpr',
       'word_count_no_emoji_hashtag_pr', 'word_count_no_emoji_hashtag_nonpr',
       'early_morning_pr', 'pr_ratio_early_morning', 'early_morning_nonpr',
       'morning_pr', 'pr_ratio_morning', 'morning_nonpr', 'afternoon_pr',
       'pr_ratio_afternoon', 'afternoon_nonpr', 'evening_pr',
       'pr_ratio_evening', 'evening_nonpr', 'late_night_pr',
       'pr_ratio_late_night', 'late_night_nonpr', 'is_weekend_pr',
       'pr_ratio_weekend', 'is_weekend_nonpr', 'is_trending_pr',
       'pr_ratio_trending', 'is_trending_nonpr', 'duration_pr',
       'duration_nonpr', 'diggCount_pr', 'diggCount_nonpr', 'shareCount_pr',
       'shareCount_nonpr', 'collectCount_pr', 'collectCount_nonpr',
       'commentCount

---

In [ ]:
# đánh giá trên linear regression, ridge, lasso ( nhớ kiểm tra đa cộng tuyến )
# thêm một file code eda và phân tích dữ liệu trong file all_kols_combined.csv